In [1]:
import os
import sys
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
import supervision as sv

# Ensure the correct Python environment is being used
print(f"Python executable: {sys.executable}")
print(f"Python version: {sys.version}")
print(f"Current working directory: {os.getcwd()}")



ModuleNotFoundError: No module named 'cv2'

In [11]:
# %%
# Install necessary packages (if not already installed)
!pip install autodistill-grounded-sam-2 supervision opencv-python-headless


/workspaces/sam2_video_analysis/notebooks/segment-anything-2
[Errno 2] No such file or directory: 'segment-anything-2'
/workspaces/sam2_video_analysis/notebooks/segment-anything-2
Obtaining file:///workspaces/sam2_video_analysis/notebooks/segment-anything-2
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Using cached tqdm-4.66.4-py3-none-any.whl.metadata (57 kB)
  Using cached hydra_core-1.3.2-py3-none-any.whl.metadata (5.5 kB)
  Using cached iopath-0.1.10-py3-none-any.whl
  Using cached omegaconf-2.3.0-py3-none-any.whl.metadata (3.9 kB)
  Using cached antlr4_python3_runtime-4.9.3-py3-none-any.whl
  Using cached portalocker-2.10.1-py3-none-any.whl.metadata (8.5 kB)
Using cached hydra_core-1.3.2-py3-none-any.whl (154 kB)
Using cached tqdm-4.66.4-py3-none-any.whl (78 kB)
Using cached omegaconf-2.3.0-py3-none-any.whl (79 kB)
Usin

In [ ]:
# Clone segment-anything-2 if not already present
if not os.path.exists('segment-anything-2'):
    !git clone https://github.com/facebookresearch/segment-anything-2
    

In [ ]:
# Install segment-anything-2
%cd segment-anything-2
!pip install -e .
%cd ..

In [ ]:
# Import necessary modules
from sam2.build_sam import build_sam2
from sam2.automatic_mask_generator import SAM2AutomaticMaskGenerator
from autodistill_grounded_sam_2 import GroundedSAM2
from autodistill.detection import CaptionOntology


In [ ]:
# Load SAM2 model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
sam2_checkpoint = f"{checkpoint_dir}/sam2_hiera_large.pt"
model_cfg = f"{checkpoint_dir}/sam2_hiera_l.yaml"
sam2 = build_sam2(model_cfg, sam2_checkpoint, device=device, apply_postprocessing=False)
mask_generator = SAM2AutomaticMaskGenerator(sam2)


In [ ]:
# Define Grounded SAM 2 with ontology
ontology = CaptionOntology({
    "basketball": "basketball",
    "number": "number on jersey",
    "court line": "court line",
    "hoop": "hoop",
    "net": "net"
})
base_model = GroundedSAM2(ontology=ontology)


In [ ]:
# Function to process a video frame by frame
def process_video(video_path):
    cap = cv2.VideoCapture(video_path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frames = []
    for _ in range(frame_count):
        ret, frame = cap.read()
        if not ret:
            break
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frames.append(frame_rgb)
    cap.release()
    return frames

In [ ]:
# Process video
video_path = 'path_to_your_video.mp4'  # Update this with your video path
frames = process_video(video_path)

# %%
# Run inference on frames
for i, frame in enumerate(frames):
    masks = mask_generator.generate(frame)
    detections = sv.Detections.from_sam(masks)
    detections.class_id = [i for i in range(len(detections))]
    mask_annotator = sv.MaskAnnotator()
    annotated_frame = mask_annotator.annotate(frame, detections)
   
    # Display the frame with annotations
    plt.figure(figsize=(8, 8))
    plt.imshow(annotated_frame)
    plt.title(f'Frame {i}')
    plt.axis('off')
    plt.show()
    
    # Optionally, save the annotated frame
    # cv2.imwrite(f'annotated_frame_{i}.png', cv2.cvtColor(annotated_frame, cv2.COLOR_RGB2BGR))
    
